In [1]:
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import datasets 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE 
from sklearn.model_selection import train_test_split
from modules import transform_fit_predict, fit_predict, print_metrics, print_time_report

%load_ext autoreload
%autoreload 2
random_state = 42

### Загрузим датасет:

In [2]:
df = datasets.load_digits()
X = df.data
y = df.target

### Создадим необходимые модели и преобразователи:

In [3]:
models = {
    'SVC': SVC(random_state=random_state),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=random_state),
    'LogisticRegression' : LogisticRegression(max_iter=10000, random_state=random_state),
}

In [4]:
transformers = {
    'PCA': PCA(n_components=0.90, random_state=random_state),
    'TSNE': TSNE(n_components=2, random_state=17)
}

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=random_state, stratify=y)

eval_data = {
    'X_train': X_train,
    'X_test': X_test,
    'y_train': y_train,
    'y_test': y_test
}

### Обучим модели и получим предикты без уменьшения размерности:

In [7]:
models_clear, predicts_clear = fit_predict(models, eval_data)

### Обучим модели и получим предикты с уменьшением размерности:

In [8]:
data = {
    'X': X,
    'y': y,

}

models_transformed, transformers, times, predicts_transformed = transform_fit_predict(models, data, transformers)

### Результаты:

In [9]:
accuracy_clear = print_metrics(predicts_clear, models_clear.keys(), y_test, message = 'Without transform')

Model SVC results Without transform :
0.9911111111111112 

Model DecisionTreeClassifier results Without transform :
0.8244444444444444 

Model LogisticRegression results Without transform :
0.9622222222222222 



In [10]:
accuracy_pca = print_metrics(predicts_transformed['PCA'], models_transformed['PCA'].keys(), y_test, transformer_name='PCA', message = 'With transform')

Model SVC results With transform PCA:
0.9866666666666667 

Model DecisionTreeClassifier results With transform PCA:
0.86 

Model LogisticRegression results With transform PCA:
0.9288888888888889 



In [11]:
accuracy_pca = print_metrics(predicts_transformed['TSNE'], models_transformed['TSNE'].keys(), y_test, transformer_name='TSNE', message = 'With transform')

Model SVC results With transform TSNE:
0.9577777777777777 

Model DecisionTreeClassifier results With transform TSNE:
0.9777777777777777 

Model LogisticRegression results With transform TSNE:
0.9066666666666666 



In [12]:
n_components_for_var =transformers['PCA'].n_components_
print(f'Для сохранения 90% дисперсии алгоритму PCA необходимо {n_components_for_var} шт. компонент')

Для сохранения 90% дисперсии алгоритму PCA необходимо 21 шт. компонент


In [13]:
print_time_report(times, 'PCA')

Алгоритм PCA:
X_train: 0.040997982025146484 сек
X_test: 0.0 сек


In [14]:
print_time_report(times, 'TSNE')

Алгоритм TSNE:
X_train: 26.926623106002808 сек


### Итоги:
#### Улучшение результатов:
- Для **DecisionTreeClassifier** и **LogisticRegression** алгоритм TSNE и PCA дает прирост метрики accuracy (TSNE лучше)
- Для **SVC** методы уменьшения размерности, наоборот, ухудшили результаты
- **SVC** без уменьшения размерности дает лучшие результаты
#### Для сохранения 90% дисперии PCA алгоритму необходимо: 
- 21 компонента 
#### Время работы алгоритмов уменьшения размерности:
- **TSNE** в сумме: 26.926623106002808 сек
- **PCA** в сумме: 0.040997982025146484 сек